# PREPROCESAMIENTO DE DATOS
1. Lectura y Formateo
2. Selección
3. Preparación

In [1]:
## Get current work directory to load all the custom modules
from pathlib import Path
import sys
BASE_DIR = Path().absolute().resolve(strict=True).parent.parent
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

In [2]:
DATA_PATH = os.path.join(BASE_DIR, 'data')
!ls ../../data/

DB_GRUAS_2021.csv  __init__.py	producto.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.impute import SimpleImputer
from matplotlib.pyplot import figure
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import re 

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import adfuller

In [5]:
from helpers.general import homogenizar_str, print_linea_de_tiempo_producto
from datetime import datetime, timedelta

># 1. Lectura y Formateo de datos

In [6]:
# Leemos los datos
df = pd.read_csv(f"{DATA_PATH}/DB_GRUAS_2021.csv", usecols=['AÑO', 'MES', 'DES_PRODUCTO', 'Cantidad Ventas'])
df.head()

,AÑO,MES,DES_PRODUCTO,Cantidad Ventas
0,2016,2,PROTECCION,0
1,2016,4,BOMBA ACODADA HDT 108 LITROS,0
2,2016,4,COUPLING ACOPLE TDFNI/UNIBOMBA,0
3,2016,4,KIT BMB.BENT AXIS HDT84 IZQ+RAC.REC,0
4,2016,4,KIT DE MONTAJE,0


In [7]:
# Cambiamos el nombre de las columnas
df.rename(columns={'DES_PRODUCTO': 'producto', 'Cantidad Ventas': 'ventas'}, inplace=True)
df.head()

,AÑO,MES,producto,ventas
0,2016,2,PROTECCION,0
1,2016,4,BOMBA ACODADA HDT 108 LITROS,0
2,2016,4,COUPLING ACOPLE TDFNI/UNIBOMBA,0
3,2016,4,KIT BMB.BENT AXIS HDT84 IZQ+RAC.REC,0
4,2016,4,KIT DE MONTAJE,0


In [8]:
# Homogenizamos los nombres de los productos
df["productoOrig"] = df.loc[:,"producto"]
df["producto"] = df["producto"].apply(homogenizar_str)
df.head()

,AÑO,MES,producto,ventas,productoOrig
0,2016,2,proteccio,0,PROTECCION
1,2016,4,bombaacodadahdt108litro,0,BOMBA ACODADA HDT 108 LITROS
2,2016,4,couplingacopletdfniunibomb,0,COUPLING ACOPLE TDFNI/UNIBOMBA
3,2016,4,kitbmbbentaxishdt84izqracre,0,KIT BMB.BENT AXIS HDT84 IZQ+RAC.REC
4,2016,4,kitdemontaj,0,KIT DE MONTAJE


In [9]:
# Ver el estado de salud de los datos
### Buscamos valores nulos 
### observamos el tipo de datos en la db
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393 entries, 0 to 2392
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   AÑO           2393 non-null   int64 
 1   MES           2393 non-null   int64 
 2   producto      2393 non-null   object
 3   ventas        2393 non-null   int64 
 4   productoOrig  2393 non-null   object
dtypes: int64(3), object(2)
memory usage: 93.6+ KB


In [10]:
# Solo valores significativos
df = df.query('ventas >= 0')
df.describe()

,AÑO,MES,ventas
count,2393.000000,2393.000000,2393.000000
mean,2018.662349,6.510238,1.170497
std,1.457477,3.541205,4.393787
min,2016.000000,1.000000,0.000000
25%,2018.000000,3.000000,0.000000
50%,2019.000000,6.000000,0.000000
75%,2020.000000,10.000000,1.000000
max,2021.000000,12.000000,96.000000


## Group by products and months
The group process going to sum all the sales in the related month and product.

In [11]:
# Agrupas datos según mes, año y producto
dfagg = df.groupby(['MES', 'AÑO', 'producto'])
dfagg = dfagg.agg({'ventas': 'sum', 'productoOrig':'first'})
dfagg = dfagg.reset_index()
dfagg = dfagg.sort_values(by=['producto', 'AÑO', 'MES'], ascending=False)
dfagg.head()

,MES,AÑO,producto,ventas,productoOrig
1957,11,2016,zsupportiinfor,0,ZSUPPORTI INFORM
615,4,2016,zsupportiinfor,0,ZSUPPORTI INFORM
754,4,2019,whasherd12s1,0,WHASHER D 1/2 S.1.5
1004,5,2019,washervolandasdesujeciondevalvuladecontrabalance,3,WASHER(VOLANDAS DE SUJECION DE VALVULA DE CONT...
753,4,2019,washervolandasdesujeciondevalvuladecontrabalance,0,WASHER(VOLANDAS DE SUJECION DE VALVULA DE CONT...


In [12]:
# seleccionando productos que se encuentran al menos en el año 2021
## de esta manera se podrán realizar predicciones en el futuro.
products21 =dfagg.query('AÑO == 2021')['producto'].unique().tolist()

># 2. Seleccion de datos

## Productos y su presencia en el tiempo

- Cada año tiene 12 meses.
- Tomaremos aquellos productos con al menos 12 meses de presencia en la línea de tiempo, es decir en el último año.
- detectamos que tenemos 8 artículos únicos que cumplen todos los requisitos

In [13]:
min_meses = 12

In [14]:
# Aplicamos el filtro para encontrar los productos significativos
temp = dfagg.loc[dfagg['producto'].isin(products21)]
temp = temp.loc[:,['producto']]
temp['count'] = 1
temp = temp.groupby('producto').count()
temp = temp.sort_values('count', ascending=False)
temp = temp.reset_index()
temp = temp.query(f'count > {min_meses}')
value_count = temp.producto.nunique()

print(f"Existen {value_count} producto que cuentan con una presencia mayor a {min_meses} meses")

Existen 8 producto que cuentan con una presencia mayor a 12 meses


In [15]:
# Tomamos solo los productos que cumples con las condiciones deseadas.
lista_articulos = temp.producto.unique().tolist()
df_clean = dfagg.loc[dfagg['producto'].isin(lista_articulos)]
df_clean = df_clean.reset_index(drop=True)
df_clean = df_clean.sort_values(by=['producto', 'AÑO', 'MES'], ascending=False)
df_clean.head()

,MES,AÑO,producto,ventas,productoOrig
0,5,2021,tuerc,7,TUERCA
1,12,2020,tuerc,5,TUERCA
2,11,2020,tuerc,1,TUERCA
3,9,2020,tuerc,11,TUERCA
4,6,2020,tuerc,4,TUERCA


### Formating datetime

In [16]:
# Creamos la columna periodo con las columnas año y mes
df_clean['Periodo'] = df_clean.apply(lambda x: datetime(year=x['AÑO'], month=x['MES'], day=1), axis=1)
df_clean.drop(columns=['AÑO', 'MES'], inplace=True)
df_clean.head()
# change columns name
df_clean.rename(columns={'producto': 'idArticulo', 'productoOrig': 'DescProducto'}, inplace=True)
df_clean.head()

,idArticulo,ventas,DescProducto,Periodo
0,tuerc,7,TUERCA,2021-05-01
1,tuerc,5,TUERCA,2020-12-01
2,tuerc,1,TUERCA,2020-11-01
3,tuerc,11,TUERCA,2020-09-01
4,tuerc,4,TUERCA,2020-06-01


In [17]:
df_clean.to_csv(f'{DATA_PATH}/producto.csv', index=False)

># 3. Preparación

### Crear series de tiempo

In [18]:
# Pivotando las tablas y llenando de ceros las ventas que no existen.
imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
df_time_pre = df_clean.pivot_table(index='Periodo', columns='idArticulo', values='ventas', aggfunc='sum',)
df_time = imp_mean.fit_transform(df_time_pre)
df_time = pd.DataFrame(df_time, columns=df_time_pre.columns, index=df_time_pre.index)
df_time.tail()

idArticulo,arandel,bateriascanrec,filtrodeair,filtrodepresio,filtroderetorn,pern,soport,tuerc
Periodo,,,,,,,,
2021-06-01,0.0,0.0,2.0,2.0,4.0,0.0,0.0,0.0
2021-07-01,0.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
2021-08-01,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2021-09-01,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2021-10-01,0.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0


### Graficando los productos en la línea de tiempo

In [19]:
print_linea_de_tiempo_producto(df_time , 'idArticulo', height=600)